## CALCULO ETo BASE

In [1]:
# import eto python library
from eto import ETo, datasets

# import pandas library
import pandas as pd

# Importacion DateTime
from datetime import datetime

# Apagamos algunos mensajes de warning
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def loading_conagua_prep_data(base_station, start_year, end_year):
    """
    Parametros
    ----------
    Entrada
        base_station : string
            Nombre de la estacion climatologica
        start_year,  : int
            Año inicial de datos conagua preparados
        end year : int
            Año Final de datos conagua preparados
    ----------
    Salida
        conagua_df : dataframe
            Dataframe que contempla todo el conjunto de datos conagua
    """
    # Cargando ubicacion de archivo con datos preparados para eto
    eto_base = '../output/exploration/conagua_'+base_station+'_'+str(start_year)+'_'+str(end_year)+'.csv'

    # Lectura de archivo con datos preparados para eto
    conagua_df = pd.read_csv(eto_base, parse_dates=True, infer_datetime_format=True, index_col='date')
    # conagua_df.columns
    # conagua_df.info()

    # Definicion de lista que contiene las columnas dentro de conagua_df
    columns_conagua_df = list(conagua_df.columns)

    # Definicion de la lista que contiene el nombre de las columnas preservar dentro de conagua_df
    columns_remaining = [
                        'e_a',
                        'T_max',
                        'T_min',
                        'RH_mean',
                        'RH_max',
                        'RH_min',
                        'P',
                        'U_z',
                        'n_sun',
                        'R_s',
                        'rain_mm'
    ]

    # A traves de la funcion set realizamos la substraccion
    # Cada elemento de columns_conagua_df se buscara dentro de columns_remaining
    # Si no se encuentra coincidencia alguna esta se guarda como elemento de columns_to_drop
    columns_to_drop_set = set(columns_conagua_df)-set(columns_remaining)
    columns_to_drop = sorted(columns_to_drop_set, key = lambda k : columns_conagua_df.index(k))

    # Removemos columns_to_drop de conagua_df y guardamos en conagua_df
    conagua_df.drop(columns=columns_to_drop,inplace=True)

    # Mostrar informacion de conagua_df
    #conagua_df.info()
    
    return conagua_df

In [3]:
def eto_base(eto, conagua_df, freq, z_msl, lat, lon, TZ_lon):
    """
    Parametros
    ----------
    Entrada
        eto : object
            Objeto de clase ETo
        freq  : string
            Frecuencia de datos contemplada 
            en base de datos conagua: hora, dia, mes
        z_msl : int
            Altitud del punto a evaluar (m)
        lat : float
            Longitud de la estacion (dec deg)
        lon : float
            Longitud de la estacion (dec deg)
        TZ_lon : Longitud central de la zona de horario (dec deg)
    ----------
    Salida
        0 : int
    """
    # Cargamos parametros base de forma que el resto puedan ser estimado automaticamente
    eto.param_est(conagua_df, freq, z_msl, lat, lon, TZ_lon)

    # Mostra informacion de los parametros
    # et0.ts_param.info()

    # Mostrando contenido de paramtros
    # display(et0.ts_param)

    # Calculando evapotranspiracion ETo referencia
    eto_ref = eto.eto_fao()

    # Agregamos columna para registrar ETo dentro de conagua_df
    conagua_df['eto'] = ''
    conagua_df['eto'] = eto_ref

    return 0

In [4]:
def export_file(conagua_df, eto):
    """
    Parametros
    ----------
    Entrada
        conagua_df : dataframe
            Dataframe Conagua + columna ETo 
        eto  : object
            Objeto de la clase ETo
    ----------
    Salida
        0 : int
    """
    # Se guardan dos archivos, el base el cual consiste de datos conagua + eto
    # y el autogenerado por eto_ts_param el cual involucra inferencia de mas variables
    conagua_df.to_csv('../output/eto/eto_'+base_station+'.csv')
    eto.ts_param.to_csv('../output/eto/eto_ts_param_'+base_station+'.csv')

    return 0

In [5]:
# Definicion de año de inicio/fin
start_year = 2019
end_year = 2020

# Definicion de base climatologica
base_station = 'cdguzman'

# Definicion de parametros
# Altura sobre el nivel del mal en metros
z_msl = 1408.99

# Altitud de la estacion (dec deg) (solo necesario si R_s or R_n no son parte de conagua_df)
lat = 19.59216667

# Longitud de la estacion (dec deg) (solo necesarios is el calculo de ETo es base horas)
lon = -103.59075

# Longitud central de la zona de horario (dec deg) (solo necesarios is el calculo de ETo es base horas)
TZ_lon = -103

# Creacion de objeto clase ETo
eto = ETo()

# Frecuencia de pandas en tipo string tanto para entradas como salidas
# Frecuencia minima es horas (H) y maxima mensual (M)
freq = 'D'

# Ejecutamos funcion de cargar datos preparados 
conagua_df = loading_conagua_prep_data(base_station, start_year, end_year)

# Ejecutamos funcion para calcular eto
eto_base(eto, conagua_df, freq, z_msl, lat, lon, TZ_lon)

# Ejecutamos funcion para exportar conagua_df +  en archivo
export_file(conagua_df, eto)